In [84]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 


In [85]:
import pandas as pd
import numpy as np

In [86]:
# Confirm folder was created and files added successfully
os.listdir("Data/")


['IMDB Movie Dataset Info.docx',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz']

## Importing title.akas

In [87]:
akas = pd.read_csv('Data/title-akas-us-only.csv', low_memory=False)



In [88]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452564 entries, 0 to 1452563
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1452564 non-null  object
 1   ordering         1452564 non-null  int64 
 2   title            1452564 non-null  object
 3   region           1452564 non-null  object
 4   language         1452564 non-null  object
 5   types            1452564 non-null  object
 6   attributes       1452564 non-null  object
 7   isOriginalTitle  1452564 non-null  object
dtypes: int64(1), object(7)
memory usage: 88.7+ MB


## Importing title.basics

In [89]:
basics = pd.read_csv('Data/title.basics.tsv.gz', sep='\t', low_memory=False)


In [90]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10017011 entries, 0 to 10017010
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 687.8+ MB


In [91]:
# Filter the basics table down to only include the US by using the filter akas dataframe
filter_us_titles = basics['tconst'].isin(akas['titleId'])
basics = basics[filter_us_titles]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
...,...,...,...,...,...,...,...,...,...
10016872,tt9916560,tvMovie,March of Dimes Presents: Once Upon a Dime,March of Dimes Presents: Once Upon a Dime,0,1963,\N,58,Family
10016901,tt9916620,movie,The Copeland Case,The Copeland Case,0,\N,\N,\N,Drama
10016939,tt9916702,short,Loving London: The Playground,Loving London: The Playground,0,\N,\N,\N,"Drama,Short"
10016962,tt9916756,short,Pretty Pretty Black Girl,Pretty Pretty Black Girl,0,2019,\N,\N,Short


In [92]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"


## Preprocessing steps for title basics

In [93]:
## changing \N to actual Nan values

In [94]:
basics = basics.replace({'\\N':np.nan})

In [95]:
## dropping Nan's from runtimeMinutes and genres columns

In [96]:
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

In [97]:
## creating filters to keep only movie in titletype, and remove documentary from genres

In [98]:
filter_titletype = basics['titleType'].str.contains('movie')
filter_documentaries = basics['genres'].str.contains('Documentary')

In [99]:
basics = basics[~filter_documentaries]

In [100]:
basics = basics[filter_titletype]

C:\Users\Admin\AppData\Local\Temp\ipykernel_12648\2716687499.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  basics = basics[filter_titletype]


In [101]:
# verifying changes

In [102]:
basics['genres'].value_counts()

Drama                        30948
Comedy                       13170
Comedy,Drama                  6332
Horror                        5535
Drama,Romance                 5479
                             ...  
Action,Mystery,War               1
Animation,Musical,Romance        1
Drama,News,Thriller              1
Fantasy,Music,Mystery            1
Biography,Fantasy,Musical        1
Name: genres, Length: 1058, dtype: int64

In [103]:
basics['titleType'].value_counts()

movie    162615
Name: titleType, dtype: int64

In [108]:
# change startYear dtype to float

In [106]:
basics['startYear'] = basics['startYear'].astype(float)

In [107]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162615 entries, 8 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          162615 non-null  object 
 1   titleType       162615 non-null  object 
 2   primaryTitle    162615 non-null  object 
 3   originalTitle   162615 non-null  object 
 4   isAdult         162615 non-null  object 
 5   startYear       159998 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  162615 non-null  object 
 8   genres          162615 non-null  object 
dtypes: float64(1), object(8)
memory usage: 12.4+ MB


In [143]:
# filter to only keep movies between startyear 2000 and 2022

In [139]:
filter_startyear = (basics['startYear'] >=2000.0) & (basics['startYear'] <=2022.0)

In [140]:
basics = basics[filter_startyear]

In [144]:
## final info check to view rows count

In [145]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86979 entries, 34802 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86979 non-null  object 
 3   originalTitle   86979 non-null  object 
 4   isAdult         86979 non-null  object 
 5   startYear       86979 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  86979 non-null  object 
 8   genres          86979 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.6+ MB


In [150]:
## export dataframe to csv in data folder

In [149]:
basics.to_csv(r'Data\basics.csv')

## Load and filter the title ratings file

In [151]:
ratings = pd.read_csv('Data/title.ratings.tsv.gz', sep='\t', low_memory=False)

In [153]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1331492 entries, 0 to 1331491
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1331492 non-null  object 
 1   averageRating  1331492 non-null  float64
 2   numVotes       1331492 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.5+ MB


In [160]:
filter_basics = ratings['tconst'].isin(basics['tconst'])
ratings = ratings[filter_basics]


In [159]:
ratings = ratings.replace({'\\N':np.nan})

In [161]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71900 entries, 17961 to 1331462
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         71900 non-null  object 
 1   averageRating  71900 non-null  float64
 2   numVotes       71900 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [162]:
ratings.to_csv(r'Data\ratings.csv')